In [ ]:
import pandas as pd
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import traceable
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os
from langchain_openai import AzureChatOpenAI

AZURE_OPENAI_API_KEY = os.environ["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
OPENAI_API_VERSION = os.environ["OPENAI_API_VERSION"]

llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=OPENAI_API_VERSION,
    deployment_name="gpt-4o-mini",
    temperature=0.1
)


In [ ]:
from pydantic import BaseModel, Field
from typing import Optional

class AgentState(BaseModel):
    user_query: str = Field(...)
    draft_answer: Optional[str] = None
    critique: Optional[str] = None
    final_answer: Optional[str] = None


In [ ]:
@traceable(name="Draft Generate Node")
def generate_draft(state: AgentState):
    response = llm.invoke(f"""
    Answer the question clearly:

    {state.user_query}
    """)
    return {"draft_answer": response.content}


In [ ]:
@traceable(name="Critic Node")
def critique_answer(state: AgentState):
    response = llm.invoke(f"""
    Critically review this answer:

    Question: {state.user_query}
    Answer: {state.draft_answer}

    Identify weaknesses and improvements.
    """)
    return {"critique": response.content}


In [ ]:
@traceable(name="Revision Node")
def revise_answer(state: AgentState):
    response = llm.invoke(f"""
    Improve the answer using the critique.

    Original Answer:
    {state.draft_answer}

    Critique:
    {state.critique}
    """)
    return {"final_answer": response.content}


In [ ]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("draft", generate_draft)
workflow.add_node("critic", critique_answer)
workflow.add_node("revise", revise_answer)

workflow.set_entry_point("draft")
workflow.add_edge("draft", "critic")
workflow.add_edge("critic", "revise")
workflow.add_edge("revise", END)

graph = workflow.compile()


In [ ]:
result = graph.invoke({
    "user_query": "Why does overfitting occur in machine learning?"
})


In [ ]:
print(result["final_answer"])